In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [85]:
#2017.5.1.
path = "/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_similarity_based-PREDICT/"
tn = pd.read_table(path + "4_5_tn_local_norm_1_1_descriptor.tsv")
no_tn = pd.read_table(path + "4_5_2_noTn_only_local_norm_1_1_descriptor.tsv")

In [84]:
#2017.4.24. independent dataset
path = "/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_similarity_based-PREDICT/"
indep = pd.read_table(path+"3_2_indep_descriptor.tsv")
indep_x = indep.values[:, 3:].astype(float)
indep_y = indep.values[:,2].astype(int)

In [3]:
f = open("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/5_final/tn+noise_1by1_idex.txt", "r")
indices = list()
while True:
    line=f.readline()
    if not line : break
    indices.append(line.lstrip('[').rstrip(']\n').split(', '))
f.close()

In [4]:
lindices = list()
for i in indices:
    tmp = list()
    for j in i:
        tmp.append(int(j))
    lindices.append(tmp)

In [78]:
len(indices[1])

1414

In [79]:
i=1
idx = lindices[i]
filtered = no_tn.filter(items = idx, axis=0) # 뽑은 index만큼 filter
noise_tn = pd.concat([tn,filtered])

x_whole_data = noise_tn[noise_tn.columns.values[3:].tolist()].values
y_whole_data = noise_tn["association"].values

In [10]:
x_whole_data

array([[ 0.80403025,  0.80403025,  0.96298073,  0.96298073],
       [ 0.99928972,  0.48549768,  0.99928972,  0.6603973 ],
       [ 0.99928972,  0.48549768,  0.99928972,  0.59343608],
       ..., 
       [ 0.99672853,  0.64241607,  0.99615848,  0.89860099],
       [ 0.98972282,  0.527713  ,  0.98972282,  0.92213089],
       [ 0.99630103,  0.4599837 ,  0.99630103,  0.98497388]])

In [38]:
try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    #plot_only = 1000
    low_dim_embs = tsne.fit_transform(x_whole_data)
    labels = y_whole_data
    plot_with_labels(low_dim_embs, labels, "training")
except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

In [36]:
low_dim_embs = tsne.fit_transform(indep_x)
labels = indep_y
plot_with_labels(low_dim_embs, labels)

In [5]:
def plot_with_labels(low_dim_embs, labels, filename='temp.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 10))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        if label==1:
            plt.scatter(x, y, color="blue", alpha=1)
        else:
            plt.scatter(x, y, color="red", alpha=0.3)
    plt.savefig("/home/share/aschoi/nas/users/asolchoi/"+filename)
    #plt.show()

In [51]:
tomat = noise_tn[['drStr_diGe', 'drStr_diPhe', 'drPhe_diGe', 'drPhe_diPhe', 'association']]

In [52]:
import scipy.io as sio
a_dict = {col_name : tomat[col_name].values for col_name in tomat.columns.values}
sio.savemat('/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/2_similarity_based(PREDICT)/1_tn_noise_1_2.mat', {'struct':a_dict})

In [53]:
a_dict

{'association': array([1, 1, 1, ..., 0, 0, 0]),
 'drPhe_diGe': array([ 0.96298073,  0.99928972,  0.99928972, ...,  0.99615848,
         0.98972282,  0.99630103]),
 'drPhe_diPhe': array([ 0.96298073,  0.6603973 ,  0.59343608, ...,  0.89860099,
         0.92213089,  0.98497388]),
 'drStr_diGe': array([ 0.80403025,  0.99928972,  0.99928972, ...,  0.99672853,
         0.98972282,  0.99630103]),
 'drStr_diPhe': array([ 0.80403025,  0.48549768,  0.48549768, ...,  0.64241607,
         0.527713  ,  0.4599837 ])}

In [54]:
tomat.to_csv('/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/2_similarity_based(PREDICT)/1_tn_noise_1_2.csv', index=False)

In [56]:
indep[['drStr_diGe', 'drStr_diPhe', 'drPhe_diGe', 'drPhe_diPhe', 'association']].to_csv('/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/2_similarity_based(PREDICT)/1_indep_1_2.csv', index=False)

In [71]:
import random
pos = tn[tn.association == 1]
tn_neg = tn[tn.association == 0]
neg = pd.concat([tn_neg, no_tn])
neg.index = range(len(neg))
pos_cnt = len(pos)
idx = random.sample(xrange(0, len(neg)), pos_cnt*2)
fitered = neg.filter(items = idx, axis=0)
result = pd.concat([pos, filtered])

In [72]:
x_whole_data = result[result.columns.values[3:].tolist()].values
y_whole_data = result["association"].values

In [67]:
try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    #plot_only = 1000
    low_dim_embs = tsne.fit_transform(x_whole_data)
    labels = y_whole_data
    plot_with_labels(low_dim_embs, labels, "training_random")
except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

In [ ]:
low_dim_embs = tsne.fit_transform(x_whole_data)
labels = y_whole_data
plot_with_labels(low_dim_embs, labels, "training_random")

In [73]:
len(y_whole_data)

2744

In [81]:
x_whole_data = tn[tn.columns.values[3:].tolist()].values
y_whole_data = tn["association"].values

In [83]:
low_dim_embs = tsne.fit_transform(x_whole_data)
labels = y_whole_data
plot_with_labels(low_dim_embs, labels, "training_tn_only")

In [86]:
temp = pd.concat([pos, no_tn])
x_whole_data = temp[temp.columns.values[3:].tolist()].values
y_whole_data = temp["association"].values

In [87]:
low_dim_embs = tsne.fit_transform(x_whole_data)
labels = y_whole_data
plot_with_labels(low_dim_embs, labels, "training_no_tn")

In [89]:
plot_with_labels(low_dim_embs, labels, "training_no_tn2")

In [90]:
path = "/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/1_feature_based-Systematic/2_PREDICT_dataset/"
f_tn = pd.read_table(path + "2_TN_local/2_normalize_indep_check/6_4_4_tn_local_norm_1_1_descriptor.tsv")
f_indep = pd.read_table(path + "2_2_indep_descriptor.tsv")

In [92]:
print len(f_tn[f_tn.association == 1])
print len(f_tn[f_tn.association == 0])

1362
1430


In [94]:
f_x_whole_data = f_tn[f_tn.columns.values[3:].tolist()].values
f_y_whole_data = f_tn["association"].values
fg_low_dim_embs = tsne.fit_transform(f_x_whole_data)
fg_labels = f_y_whole_data
plot_with_labels(fg_low_dim_embs, fg_labels, "feature_based_training.png")

In [93]:
f_indep_x = f_indep.values[:, 3:].astype(float)
f_indep_y = f_indep.values[:,2].astype(int)
fi_low_dim_embs = tsne.fit_transform(f_indep_x)
fi_labels = f_indep_y
plot_with_labels(fi_low_dim_embs, fi_labels, 'feature_based_indep.png')

In [4]:
# 2017.6.2.
under = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/data/desc_12/under.tsv")

In [6]:
x_whole_data = under[under.columns.values[3:].tolist()].values
y_whole_data = under["association"].values

In [7]:
try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    #plot_only = 1000
    low_dim_embs = tsne.fit_transform(x_whole_data)
    labels = y_whole_data
    plot_with_labels(low_dim_embs, labels, "underunder")
except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [9]:
neg = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/data/desc_12/gold_neg_desc12.tsv")
pos = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/data/desc_12/gold_pos_desc12.tsv")

In [11]:
concated = pd.concat([neg, pos])

In [20]:
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/data/desc_12/indep_desc12.tsv")
indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

In [14]:
x_whole_data = concated[concated.columns.values[3:].tolist()].values
y_whole_data = concated["association"].values

In [17]:
from imblearn.under_sampling import RandomUnderSampler
# undersampler
rus = RandomUnderSampler(ratio=0.5)
x_resampled, y_resampled = rus.fit_sample(x_whole_data, y_whole_data)

In [18]:
low_dim_embs = tsne.fit_transform(x_resampled)
labels = y_resampled
plot_with_labels(low_dim_embs, labels, "under2")

In [21]:
low_dim_embs = tsne.fit_transform(indep_x)
labels = indep_y
plot_with_labels(low_dim_embs, labels, "indep_12")